In [5]:

import torch
import matplotlib.pyplot as plt
import logging

torch.backends.cudnn.benchmark = True

logging.basicConfig(filename='training_2048.log', filemode='a', level=logging.INFO, format='%(asctime)s %(message)s')
logging.info('Starting training')

from core.evaluation.mcts_hypers import MCTSHypers
from core.resnet import TurboZeroResnet, TurboZeroArchParams
from core.training.training_hypers import TurboZeroHypers
from envs._2048.evaluator import _2048MCTS
from envs._2048.trainer import _2048Trainer, load_checkpoint


In [6]:
NUM_PARALLEL_ENVS = 100
CHECKPOINT_FILE = ''
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEBUG = True # disables JIT compilation

In [7]:
if CHECKPOINT_FILE:
    trainer = load_checkpoint(NUM_PARALLEL_ENVS, CHECKPOINT_FILE, DEVICE, debug=DEBUG)
else:
    arch_params=TurboZeroArchParams(
        input_size=torch.Size((1, 4, 4)),
        policy_size=4,
        res_channels=4,
        res_blocks=1,
        value_head_res_channels=4,
        value_head_res_blocks=1,
        policy_head_res_channels=4,
        policy_head_res_blocks=1,
        kernel_size=3,
        value_output_activation=torch.nn.Tanh()
    )

    hypers=TurboZeroHypers()

    eval_hypers = MCTSHypers(num_iters = 100)

    run_tag='test_2048_1'

    model = TurboZeroResnet(arch_params).to(DEVICE)

    optimizer = torch.optim.AdamW(model.parameters(), lr=hypers.learning_rate)

    trainer = _2048Trainer(
        _2048MCTS(NUM_PARALLEL_ENVS, DEVICE, eval_hypers, debug=DEBUG),
        _2048MCTS(hypers.test_episodes_per_epoch, DEVICE, eval_hypers, debug=DEBUG),
        NUM_PARALLEL_ENVS,
        DEVICE,
        torch.device('cpu'),
        model,
        optimizer,
        hypers,
        run_tag=run_tag
    )

plt.close('all')

In [ ]:
trainer.training_loop()